In [11]:
#
# importing the necessary libraries: 
# NumPy, SciPy (spatial algorithms), timing functions, a handler for csv files 
#
import numpy as np
import scipy.spatial as sp
import time
import csv

In [12]:
#
# Path to the pics storge directory
#
file_dir = "/Applications/SageMath/icp_init/pics/"

In [13]:
#
# Centering a point cloud A
#
def barycentered(A):
    #
    bar = np.sum(A, axis=1)/A.shape[1]
    #
    return A - bar[:, np.newaxis]

In [14]:
#
# Best orthogonal fit between two (centred) labelled point clouds
#
# A and B are assumed to have the same number of points
#
def best_fit(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na==nb
    #
    H = A @ B.T
    W, S, V = np.linalg.svd(H)
    #
    return V.T @ W.T

In [15]:
#
# Detecting nearest neighbours between two unlabelled point coluds A and B
#
# Assumption: cardinality of A <= cardinality of B 
#
# Input: point clouds A, B
#
# Output: for each point of A a matching point of B, as a 0/1-matrix
#
def nearest_neighbours(A, B):
    #
    na = A.shape[1]
    nb = B.shape[1]
    #
    assert na <= nb
    #
    tree = sp.KDTree(A.T, leafsize=10, compact_nodes=True, copy_data=True, balanced_tree=True)
    #
    matching = []
    I = matrix.identity(na)
    #
    for i in range(nb):
        _, ind = tree.query(B.T[i], k=1, p=2, workers=-1)
        matching += [I[ind]]
    #    
    return np.array(matching)

In [16]:
#
# ICP algorithm for two unlabelled point clouds A and B
#
# Assumption: cardinality of A <= cardinality of B
#
# Input:
# A, B = unlabelled point clouds
# init = initial orthogonal transformation
# max_iter = maximum number of iterations
# tol = tolerance for halting the computation
#
# Output:
# U = orthogonal transformation bringing A to B as close as possible
# nn = nearest neighbor matching between U*A and B
# dist = distance between U*A and B*nn (in the max singular value norm)
#
def icp(A, B, init=None, max_iter=100, tol=1e-16):
    #
    src = copy(A)
    dst = copy(B)
    #
    na = src.shape[1]
    nb = src.shape[1]
    #
    assert na <= nb
    #
    if (init is not None):
        src = init @ src
    #    
    prev_err = 0
    #
    for i in range(max_iter):
        #   
        nn = nearest_neighbours(src, dst)
        U = best_fit(src, dst @ nn)
        #    
        src = U @ src
        #
        diff = src - dst @ nn
        err  = np.linalg.norm(diff, 2)
        #
        if abs(prev_err - err) < tol:
            break
        #    
        prev_err = err
    #
    U = best_fit(A, src)
    nn = nearest_neighbours(U @ A, dst)
    #
    diff = U @ A - B @ nn
    dist = np.linalg.norm(diff, 2)
    #
    return U, nn, dist

In [17]:
#
# Testing ICP without initialization on a given point cloud 
#
# Input: a point cloud P
#
# Generates a random O and S, and produces Q = O*P*S, then applies the Main Algorithm
#
# Output: flag = True if o = O and s = S are successfully recovered, flag = False otherwise
#
# Prints out o, s, and checks their proximity to O, S, respectively
# Prints the distance between Q and o*P*s in the max singular value norm
#
def test_point_cloud_no_init(P, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q = O @ P @ S
    Q = barycentered(Q)
    #
    o_no_init, s_no_init, dist_no_init = icp(P, Q, None)
    #
    flag_no_init = np.allclose(dist_no_init, 0)
    #
    diff = o_no_init - O
    dist_ortho_no_init = np.linalg.norm(diff, 2)
    diff = s_no_init.T - S
    dist_per_no_init = np.linalg.norm(diff, 'fro')**2/(2*num)
    #
    if verbose:
        print("Orthogonal transformation found:")
        print(o_no_init)
        print("Distance to the initial one?")
        print(dist_ortho_no_init)
        print("Permutation found:")
        print(s_no_init.T)
        print("Hamming distance to the initial one?")
        print(dist_per_no_init)
        print("Distance to image:")
        print(dist_no_init)
    #
    return flag_no_init, dist_ortho_no_init, dist_per_no_init, dist_no_init

In [18]:
#
# Testing the Main Algorithm (ICP with initialization) on a given point cloud 
#
# Input: a point cloud P
#
# Generates a random O and S, and produces Q = O*P*S, then applies the Main Algorithm
#
# Output: flag = True if o = O and s = S are successfully recovered, flag = False otherwise
#
# Prints out o, s, and checks their proximity to O, S, respectively
# Prints the distance between Q and o*P*s in the max singular value norm
#
def test_point_cloud_init(P, verbose=False):
    #
    dim = P.shape[0]
    num = P.shape[1]
    if verbose:
        print("Number of points: {}".format(num))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    P = barycentered(P)
    #
    Q = O @ P @ S
    Q = barycentered(Q)
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    assert np.allclose(O @ Ep @ O.T - Eq, np.zeros([dim,dim]))
    assert np.allclose(U0 @ Ep @ U0.T - Eq, np.zeros([dim,dim]))
    assert(np.allclose(Eigp, Eigq))
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    flag_init = False
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        o_init, s_init, dist_init = icp(P, Q, U)
        flag_init = flag_init or np.allclose(dist_init, 0)
        if flag_init:
            break
    #
    diff = o_init - O
    dist_ortho_init = np.linalg.norm(diff, 2)
    diff = s_init.T - S
    dist_per_init = np.linalg.norm(diff, 'fro')**2/(2*num)
    #
    if verbose:
        print("Orthogonal transformation found:")
        print(o_init)
        print("Distance to the initial one?")
        print(dist_ortho_init)
        print("Permutation found:")
        print(s_init.T)
        print("Hamming distance to the initial one?")
        print(dist_per_init)
        print("Distance to image:")
        print(dist_no_init)
    #
    return flag_init, dist_ortho_init, dist_per_init, dist_init

In [19]:
#
# Running a given number of tests num_tests on various point clouds 
# generated by calling test_point_cloud_no_init(P, verbose) and then
# test_point_cloud_init(P, verbose)
#
# Some statistics is collected (time elapsed, test success rate, etc)
#
def run_tests_point_cloud(P, num_tests, verbose=False):
    #
    num_success_no_init = 0
    num_fail_no_init  = 0
    rec_dist_no_init = []
    rec_dist_ortho_no_init = []
    rec_dist_per_no_init = []
    #
    num_success_init = 0
    num_fail_init = 0
    rec_dist_init = []
    rec_dist_ortho_init = []
    rec_dist_per_init = []
    #
    start = time.process_time()
    #
    for i in range(num_tests):
        #
        msg = '### Test #{} : '.format(i+1)
        if verbose:
            print(msg)
        #
        flag_no_init, dist_ortho_no_init,\
        dist_per_no_init, dist_no_init = test_point_cloud_no_init(P, verbose)
        #
        rec_dist_ortho_no_init += [dist_ortho_no_init]
        rec_dist_per_no_init += [dist_per_no_init]
        rec_dist_no_init += [dist_no_init]
        #
        if flag_no_init:
            num_success_no_init += 1
        else:
            num_fail_no_init += 1
        #
        flag_init, dist_ortho_init,\
        dist_per_init, dist_init = test_point_cloud_init(P, verbose)
        #
        rec_dist_ortho_init += [dist_ortho_init]
        rec_dist_per_init += [dist_per_init]
        rec_dist_init += [dist_init]
        #
        if flag_init:
            num_success_init += 1
        else:
            num_fail_init += 1
        #
        msg = msg + 'NO INIT: SUCCESS {}'.format(num_success_no_init)
        msg = msg + ' FAIL {};'.format(num_fail_no_init)
        msg = msg + ' WITH INIT: SUCCESS {}'.format(num_success_init)
        msg = msg + ' FAIL {}'.format(num_fail_init)
        print(msg, end='\r')
    #   
    end = time.process_time()
    #
    assert (num_success_no_init + num_fail_no_init == num_tests)
    assert (num_success_init + num_fail_init == num_tests)
    #
    avg_dist_ortho_no_init = np.mean(rec_dist_ortho_no_init)
    avg_dist_per_no_init = np.mean(rec_dist_per_no_init)
    avg_dist_no_init = np.mean(rec_dist_no_init)
    #
    avg_dist_ortho_init = np.mean(rec_dist_ortho_init)
    avg_dist_per_init = np.mean(rec_dist_per_init)
    avg_dist_init = np.mean(rec_dist_init)
    #
    print("Time elapsed:", time.strftime('%H:%M:%S', time.gmtime(end-start)), ' '*55)
    print("Success rate *without* initialization:", float(num_success_no_init/num_tests))
    print("======= Mean values =======")
    print("Distance to initial orthogonal transformation: {}".format(avg_dist_ortho_no_init))
    print("Hamming distance to initial permutation (normalised): {}".format(avg_dist_per_no_init))
    print("Distance to image: {}".format(avg_dist_no_init))
    #
    print("Success rate *with* initialization:", float(num_success_init/num_tests))
    print("======= Mean values =======")
    print("Distance to initial orthogonal transformation: {}".format(avg_dist_ortho_init))
    print("Hamming distance to initial permutation (normalised): {}".format(avg_dist_per_init))
    print("Distance to image: {}".format(avg_dist_init))
    #

In [20]:
#
# Running tests for comparison: ICP without and with initialization
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
# First, we run ICP without initialization, in which case is almost surely fails. 
# Then we run ICP with prior initialization initialization, in which case it succeeds. 
#
# Otput: some statistics on the test parameters
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) for v in line]]
    #    
    f.close()
    #
    P = np.array(P).T
    #
    print("File read: {}".format(name))
    #
    run_tests_point_cloud(P, num_tests=100, verbose=False)
    #
    print("#"*42)

File read: Teapot.csv
Time elapsed: 01:02:22                                                        
Success rate *without* initialization: 0.0
======= Mean values =======
Distance to initial orthogonal transformation: 1.8660295159359506
Hamming distance to initial permutation (normalised): 0.9978632478632479
Distance to image: 4.982988638159862
Success rate *with* initialization: 1.0
======= Mean values =======
Distance to initial orthogonal transformation: 9.422373688206196e-16
Hamming distance to initial permutation (normalised): 0.0
Distance to image: 1.9593842270698704e-15
##########################################
File read: Bunny.csv
Time elapsed: 02:34:42                                                        
Success rate *without* initialization: 0.02
======= Mean values =======
Distance to initial orthogonal transformation: 1.8974596187079795
Hamming distance to initial permutation (normalised): 0.9783522727272727
Distance to image: 5.496200430928091
Success rate *with* init

In [21]:
#
# Generating some pictures for comparison: ICP without and with initialization
#
# The following unoclluded point clouds are used: teapot, bunny, cow. 
# First, we run ICP without initialization, in which case is almost surely fails. 
# Then we run ICP with prior initialization initialization, in which case it succeeds. 
#
# Otput: 
#
# First, an image with non-initialized ICP applied. 
# Specimen = blue. Image = red. Recovered image (ICP matching specimen to image) = green. 
#
# Second, an image with initialized ICP applied. 
# Specimen = blue. Image = red. Recovered image (ICP matching specimen to image) = green.
#
# Since the matching is almost perfect, red points may not be visible as they are all
# covered by the green points of the recovered image
#
#
filenames = ["Teapot.csv", "Bunny.csv", "Cow.csv"]
#
for name in filenames:
    #
    P = []
    shift = [0.3, 0.4, 0.5]
    #
    f = open(name)
    reader = csv.reader(f)
    #
    for line in reader:
        P += [[RDF(v) + u for (v, u) in zip(line, shift)]]
    #    
    f.close()
    #    
    P = np.array(P).T
    #
    dim = P.shape[0]
    num = P.shape[1]
    #
    print("File read: {}".format(name))
    #
    seed = np.random.normal(0.0, 1.0, (dim, dim))
    O = np.linalg.qr(seed, mode='complete')[0]
    #
    S = np.random.default_rng().permutation(np.identity(num))
    #
    Q = O @ P @ S
    #
    o_no_init, s_no_init, d_no_init = icp(P, Q, None)
    #
    R_no_init = o_no_init @ P
    #
    Ep = P @ P.T
    Eigp, Up = np.linalg.eigh(Ep)
    #
    Eq = Q @ Q.T
    Eigq, Uq = np.linalg.eigh(Eq)
    #
    U0 = Uq @ Up.T
    #
    assert np.allclose(O @ Ep @ O.T - Eq, np.zeros([dim,dim]))
    assert np.allclose(U0 @ Ep @ U0.T - Eq, np.zeros([dim,dim]))
    assert(np.allclose(Eigp, Eigq))
    #
    isoms_discrete = MatrixGroup([matrix.diagonal(d) for d in Permutations([-1]+[1]*(dim-1))])
    isoms_discrete = [np.array(matrix(m)) for m in isoms_discrete]
    #
    flag = False
    for isom in isoms_discrete:
        U = U0 @ Up @ isom @ Up.T
        o_init, s_init, dist_init = icp(P, Q, U)
        flag = flag or np.allclose(dist_init, 0)
        if flag:
            R_init = o_init @ P
            break
    #
    pointsP = point3d(P.T, size=2, color='blue')
    pointsQ = point3d(Q.T, size=2, color='red')
    pointsR_no_init = point3d(R_no_init.T, size=2, color='green')
    pointsR_init = point3d(R_init.T, size=2, color='green')
    #
    img_no_init = pointsP + pointsQ + pointsR_no_init
    p_no_init = img_no_init.plot()
    #
    img_init = pointsP + pointsQ + pointsR_init
    p_init = img_init.plot()
    #
    t = time.process_time()
    #
    p_no_init.save(file_dir+"comparison_no_init_{}.html".format(t))
    p_init.save(file_dir+"comparison_init_{}.html".format(t))
    #
    print("Images saved ...")

File read: Teapot.csv
Images saved ...
File read: Bunny.csv
Images saved ...
File read: Cow.csv
Images saved ...
